In [23]:
# importing the required libraries
from PyPDF2 import PdfReader
import docx2txt
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import pandas as pd
import os
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
# Load the trained spaCy BERT model from the specified path
nlp_bert = spacy.load('/home/neha/workspace/neha--BERT/BERT_model_test/Fine_tune_BERT_with_spacy3/model-best')

# initializing the countvectorizer
cv = CountVectorizer()

# Initializing the Sentence Transformer model using BERT with mean-tokens pooling for embeddings creation
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [44]:
# function to get the from the text extracted
def get_text_labels(text):
    doc_bert = nlp_bert(text)
    text_corpus = []
    label_corpus = []
    # Iterate through the named entities (entities) recognized by the model
    for ent in doc_bert.ents:
    # Print the recognized text and its corresponding label
    #   print(ent.text, "  ->>>>  ", ent.label_)
        text_corpus.append(ent.text)
        label_corpus.append(ent.label_)
    # print("/nPrinting the length of skill words in the resume/n")
    # print(len(text_corpus))    

    # cleaning the text_corpus    
    text_corpus_cleaned = []
    for sub in text_corpus:
        text_corpus_cleaned.append(sub.replace("\n",""))
    # print(text_corpus_cleaned)
    # print("\n")


    required_text = []
    for ele in range(len(label_corpus)):
        if label_corpus[ele]=='DEGREE' or label_corpus[ele]=='SKILLS' or label_corpus=='DIPLOMA_MAJOR' or label_corpus=='EXPERIENCE':
                required_text.append(text_corpus_cleaned[ele])
   # print(required_text)
            
    # converting the text_corpus to string
    skills_corpus = ' '.join(map(str, required_text))
    return skills_corpus        

In [4]:
# function to extract text from docx files
def extract_text_from_docx(docx_path):
    text = docx2txt.process(docx_path)
    return text

In [5]:
# function to extract text from pdf files
def extract_text_from_pdf(pdf_file_path):
    reader = PdfReader(pdf_file_path)
    number_of_pages = len(reader.pages)
    page = reader.pages[0]
    text = page.extract_text()
    return text

In [6]:
# def extract_text_from_txt(txt_file_path):
#     with open(txt_file_path) as f:
#         contents = f.read()
#     return contents

In [7]:
# function to check the file type and calls the extract function to get the text from various resume files
def check_file_type(name_of_file):# considered two file formats - 1.pdf , 2.docx
    if name_of_file.endswith('.pdf'):
        text = extract_text_from_pdf(name_of_file)
    elif name_of_file.endswith('.docx'):
        text = extract_text_from_docx(name_of_file)
    # elif name_of_file.endswith('.txt'):
    #     text  = extract_text_from_txt(name_of_file)
    skills_extracted = get_text_labels(text)

    return skills_extracted

In [8]:
# initializing the punctuations ,stopswords and wordlematizer
punctuations_ = string.punctuation
stop_words = set(stopwords.words('english'))
stop_words.add('https')
lematizer = WordNetLemmatizer()

In [9]:
# text preprocessing for the job_description
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])


def lematize_words(text):
    return " ".join([lematizer.lemmatize(word) for word in text.split()])


def text_cleaning(text):
    text = text.lower()
    for words in punctuations_:
        text = text.replace(words,"")
    text = " ".join(text.split('\n'))
    text = remove_stopwords(text)
    text  = lematize_words(text)
    text = re.sub(' +',' ',text)
    return text

In [40]:
# function to calculate the matching score    
def score_calculation_cv(skills_corpus,jd_require):
    test_pair = [skills_corpus,jd_require]
    count_matrix = cv.fit_transform(test_pair)
    # print("\nSimilarity Scores:")
    # print(cosine_similarity(count_matrix))
    # print("--------------------------------------")
    # print("--------------------------------------")
    matchPercentage = cosine_similarity(count_matrix)[0][1] * 100
    matchPercentage = round(matchPercentage, 2) # round to two decimal
    return matchPercentage

In [42]:
def score_calculation_bert(skills_corpus,jd_require):
       
        # Encoding the sentences to obtain their embeddings
        skills_embeddings = model.encode(skills_corpus)
        jd_embeddings  = model.encode(jd_require)

        # Calculating the cosine similarity between the first sentence embedding and the rest of the embeddings
        # The result will be a list of similarity scores between the first sentence and each of the other sentences
        similarity_scores = cosine_similarity(skills_embeddings, jd_embeddings)
        # print(similarity_scores)
        matchPercentage = similarity_scores[0][0]*100
        matchPercentage = round(matchPercentage,2)
        return matchPercentage

In [13]:
with open('/home/neha/workspace/Resume_selector/job_requirements.txt') as f:
    contents = f.read()
jd_require = text_cleaning(contents)    

In [31]:
def resume_rank(resume_names,result_cv,result_bert):

    dict_cv= {"Name":resume_names,"Score_CV":result_cv}
    df1  = pd.DataFrame(dict_cv)
    dict_bert = {"Name":resume_names,"Score_BERT":result_bert}
    df2 = pd.DataFrame(dict_bert)
    test1 = df1.sort_values('Score', ascending=False)
    test2 = df2.sort_values('Score',ascending=False)
    print("\nDisplaying the rank using Countvectorizer\n")
    print(test1)
    print("\nDisplaying the result using BERT\n")
    print(test2)


In [46]:
result_cv = []
result_bert = []
resume_names = []

test_folder_path = r"/home/neha/workspace/Final/test/"
for file in os.listdir(test_folder_path):
    # print(file)
    resume_names.append(file)
    resume_file_path = test_folder_path+file
    skills_corpus = check_file_type(resume_file_path)
    skills_corpus = text_cleaning(skills_corpus)
    score1 = score_calculation_cv(skills_corpus,jd_require)
    score2 = score_calculation_bert([skills_corpus],[jd_require])
    result_cv.append(score1)
    result_bert.append(score2)
resume_rank(resume_names,result_cv,result_bert)


Displaying the rank using Countvectorizer

                            Name  Score
0                    Hasibul.pdf   6.00
1                  PRITI PAL.pdf   5.30
2            Dipakshi Kumari.pdf   4.35
4  John Benedict L. Soledad .pdf   1.23
3                  Cara Chiu.pdf   0.00

Displaying the result using BERT

                            Name  Score
0                    Hasibul.pdf  61.49
4  John Benedict L. Soledad .pdf  58.20
1                  PRITI PAL.pdf  57.68
2            Dipakshi Kumari.pdf  54.84
3                  Cara Chiu.pdf  49.16


In [1]:
# result_bert[0][0][0]

In [2]:
# result_cv

In [3]:
# for value in result:
#     print(value)
#     if str(value)=='None':
#         result.remove(value)

In [4]:
# result

In [5]:
# dict= {"Name":resume_names,"Score":result}
# df  = pd.DataFrame(dict)

In [6]:
# print(df)

In [7]:
# test = df.sort_values('Score', ascending=False)

In [8]:
# test